In [3]:
import pickle
from itw_tools import get_demands, no_date_gaps, get_forecasts
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
# how many previous datapoint to check
lookback = 30

In [4]:
# get the data
with open("testdata.pkl", "rb") as f:
    d = pickle.load(f)
e = d[0]
#dvtest=d[1]
p, tp, d = get_demands(e)
pfc, tpfc, fc = get_forecasts(e)
timeseries = np.array(d).astype('float32')

In [5]:
# reshaping to get in the format (N rows, 1 Column )
# -1 means number of rows will be automatically decided by Puthon
timeseries = timeseries.reshape(-1,1)

In [6]:
def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset

    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    for i in range(len(dataset)-lookback):
        feature = dataset[i:i+lookback]
        target = dataset[i+lookback]  #dataset[i+1:i+lookback+1]
        X.append(feature)
        y.append(target)
        
    X = np.array(X)
    y = np.array(y)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [7]:
# normalize the data to bring into common range of 0-1 to converse the model
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
timeseries = sc.fit_transform(timeseries)

In [8]:
# get X and y
X, y = create_dataset(timeseries, lookback=lookback)

In [9]:
# split data in test and training cases
train_size = int(len(y) * 0.80)

In [10]:
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [11]:
X_train.shape
X_test.shape
# reshaping to match RNN model input format
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [12]:
X_train.shape
X_test.shape

torch.Size([77, 1, 30])

In [13]:
train_loader = DataLoader(TensorDataset(X_train, y_train), \
                                  batch_size = lookback, shuffle = True)
test_loader = DataLoader(TensorDataset(X_test, y_test), \
                                 batch_size = lookback, shuffle = False)

# CODE

In [14]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define your CNN-LSTM model
class CNNLSTM(nn.Module):
    def __init__(self, input_shape, num_classes, params):
        super(CNNLSTM, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(input_shape[0], params['conv1_filters'], kernel_size=params['conv1_kernel_size'], padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(params['conv1_filters'], params['conv2_filters'], kernel_size=params['conv2_kernel_size'], padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.lstm = nn.LSTM(input_size=params['lstm_input_size'], hidden_size=params['lstm_hidden_size'], num_layers=params['lstm_layers'], batch_first=True)
        self.fc = nn.Linear(params['lstm_hidden_size'], num_classes)

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), x.size(1), -1)  # Flatten for LSTM
        _, (x, _) = self.lstm(x)
        x = self.fc(x[-1])  # Use the last LSTM output
        return x

# Define the objective function for Optuna
def objective(trial):
    # Define hyperparameters to optimize
    params = {
        'conv1_filters': trial.suggest_int('conv1_filters', 16, 64),
        'conv1_kernel_size': trial.suggest_int('conv1_kernel_size', 3, 5),
        'conv2_filters': trial.suggest_int('conv2_filters', 16, 64),
        'conv2_kernel_size': trial.suggest_int('conv2_kernel_size', 3, 5),
        'lstm_input_size': trial.suggest_int('lstm_input_size', 64, 256),
        'lstm_hidden_size': trial.suggest_int('lstm_hidden_size', 64, 256),
        'lstm_layers': trial.suggest_int('lstm_layers', 1, 3)
    }
    
    # Create the model
    model = CNNLSTM(input_shape=(3, 32, 32), num_classes=10, params=params)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train the model
    for epoch in range(5):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
    # Evaluate the model on the validation set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    
    return accuracy

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Get the best hyperparameters
best_params = study.best_params
best_accuracy = study.best_value


ModuleNotFoundError: No module named 'optuna'

In [15]:
! pip install -q optuna

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\avitr\\anaconda3\\lib\\site-packages\\PyYAML-5.4.1.dist-info\\METADATA'

